In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import display,HTML
import pandasql as ps 

class Web_Scrapper:

    def __init__(self) -> None:
        pass

# crea lista de urls
    def list_maker_attribute(self,repeticion,lista,tag_name,attribute) -> list:
        """Crea una lista de valores de atributos, extrae elementos de otra lista identidicados por TAG_NAME, util para obtener URL. 

        Args:
            repeticion (_int_): Numero de veces que se itera
            lista (_list_): Lista que contiene los elementos html
            tag_name (_str_): Localizador tag (html)
            attribute (_str_): Atributo del que se desea obtener valor

        Returns:
            list: Lista de los valores del atributo deseado
        """
        lista_final=[]
        for i in range(repeticion):
            try:
                lista_final.append(lista[i].find_element(By.TAG_NAME,tag_name).get_attribute(attribute))
            except Exception:
                lista_final.append(np.nan)
        return lista_final

# crea lista de un elemento html especifico indicado por class name
    def list_maker_class(self,repeticion,lista,class_name) -> list:
        """Crea una lista de valores de atributos, extrae elementos de otra lista identidicados por CLASS_NAME, util para obtener contenido de elementos html. 

        Args:
            repeticion (_int_): Numero de veces que se itera
            lista (_list_): Lista que contiene los elementos html
            class_name (_str_): Valor del atributo class (html)

        Returns:
            list: Lista de los valores del atributo deseado (class)
        """
        lista_final=[]
        for i in range(repeticion):
            try:
                lista_final.append(lista[i].find_element(By.CLASS_NAME,class_name).text)
            except Exception:
                lista_final.append(np.nan)
        return lista_final    
    
# crea lista de un elemento html especifico indicado por class name dentro de un class name
    def list_maker_class2(self,repeticion,lista,class_name,class_name2) -> list:
        """Crea una lista de valores de atributos, extrae elementos de otra lista identidicados por CLASS_NAME dos ocasiones, util para obtener contenido de elementos html. 

        Args:
            repeticion (_int_): Numero de veces que se itera
            lista (_list_): Lista que contiene los elementos html
            class_name (_str_): Valor del atributo class (html)
            class_name2 (_str_): Valor del segundo atributo class (html)

        Returns:
            list: Lista de los valores del atributo deseado (class)
        """
        lista_final=[]
        for i in range(repeticion):
            try:
                lista_final.append(lista[i].find_element(By.CLASS_NAME,class_name).find_element(By.CLASS_NAME,class_name2).text)
            except Exception:
                lista_final.append(np.nan)
        return lista_final

    def delete_extra_info(self,lista,caracter) -> list:
        """Extrae substring de un string con metodo split, conserva la particion de interes.

        Args:
            lista (_list_): Lista con elementos que se quiere reducir
            caracter (_str_): Cadena a partir de la cual se haran particiones

        Returns:
            list: Lista con elementos reducidos
        """
        for i in range(len(lista)):
            lista[i]=lista[i].split(caracter)[0]
        return lista

    def to_binary(self,lista,condicion) -> list:
        """A partir de una condicion, sustituye por 1 si la condicion es cierta, falso en otro caso para los elementos de una lista.

        Args:
            lista (_list_): Lista sin estandarizar
            condicion (_str_): Condicion que se tomara como verdadera (1)

        Returns:
            list: Lista con elementos 1 y 0, segun cumplan la condicion
        """
        for i in range(len(lista)):
            lista[i] = 1 if lista[i]==condicion else 0
        return lista

    def df_to_excel(self,dataframe,tienda,producto): #importa un dt a excel
        """Exporta un dataframe a un archivo .xlsx

        Args:
            dataframe (_dataframe_): Dataframe que se desea exportar
            tienda (_str_): Nombre de la tienda asociada al dataframe
            producto (_str_): Producto asociado al dataframe
        """
        dataframe.to_excel(f"scrapper_{tienda}_{producto}.xlsx")

    def imagen(self,path): #visualiza imagen (con url imagen)
        """"Ayuda a visualizar la URL como imagen.

        Args:
            path (_str_): URL de la imagen

        Returns:
            _str_: Representa elemento html
        """
        return f'<img src="{path}" width="250" >'
    

    def scrapping_walmart(self,producto:str) -> pd:
        """Web scrapper para el sitio de Walmart.

        Args:
            producto (str): producto del que se desea obtener dataframe

        Returns:
            pd: dataframe del producto
        """
        driver = webdriver.Chrome(service=Service("C://Users//basal//Downloads//chromedriver_win32//chromedriver.exe"))
        driver.implicitly_wait(15)
        producto=producto.replace(" ","%20")
        driver.get(f"https://www.walmart.com.mx/productos?Ntt={producto}")
        driver.implicitly_wait(15)
        productos=driver.find_elements(By.CLASS_NAME,"product_container__3DSm4")

        lista_nombres=self.list_maker_class(len(productos),productos,"nav-link_navLink__2oJ29.product_name__1YFfY")
        lista_marcas=self.list_maker_class(len(productos),productos,"product_brand__1CsRr")
        lista_url_imagenes=self.list_maker_attribute(len(productos),productos,"img","src")
        lista_urls_productos=self.list_maker_attribute(len(productos),productos,"a","href")
        lista_precios=self.list_maker_class2(len(productos),productos,"product_price__2NBjj","text_text__3oq-D.text_large__K4EIS.text_bold__2Ptj-")

        data={"MARCA":lista_marcas,"PRODUCTO":lista_nombres,"PRECIO":lista_precios,"IMAGEN":lista_url_imagenes,"LINK":lista_urls_productos}
        df_walmart=pd.DataFrame(data)
        df_walmart.insert(0,"FECHA",datetime.now().strftime("%d/%m/%Y"))
        df_walmart.insert(1,"TIENDA","WALMART")
        df_walmart["PRECIO"]=df_walmart["PRECIO"].str.replace("$"," ")

        self.df_to_excel(df_walmart,"walmart",producto)

        display(HTML(df_walmart.to_html(escape=False,formatters=dict(IMAGEN=self.imagen))))



    def scrapping_claro(self,producto:str) -> pd:
        """Web scrapper para el sitio de Claro Shop.

        Args:
            producto (str): producto del que se desea obtener dataframe

        Returns:
            pd: dataframe del producto
        """
        driver = webdriver.Chrome(service=Service("C://Users//basal//Downloads//chromedriver_win32//chromedriver.exe"))
        driver.implicitly_wait(5)
        producto=producto.replace(" ","%20")
        driver.get(f"https://www.claroshop.com/resultados/q={producto}/pagina=1")
        driver.implicitly_wait(5)
        productos=driver.find_elements(By.CLASS_NAME,"cardProduct")

        lista_nombres=self.list_maker_class2(len(productos),productos,"contDataCard","h4")
        lista_urls_productos=self.list_maker_attribute(len(productos),productos,"a","href")
        lista_precios=self.list_maker_class2(len(productos),productos,"contDataCard","precio1")
        lista_precios_limpia=self.delete_extra_info(lista_precios," ") #Eliminamos info extra de la lista

        lista_envios=self.list_maker_class2(len(productos),productos,"expressFreeCont","envioGratis")
        lista_envios_binaria=self.to_binary(lista_envios,"Envío gratis") #Envio gratis = 1
        

        data={"PRODUCTO":lista_nombres,"PRECIO":lista_precios,"ENVIO":lista_envios_binaria,"LINK":lista_urls_productos}
        df_claro=pd.DataFrame(data)
        df_claro.insert(0,"FECHA",datetime.now().strftime("%d/%m/%Y"))
        df_claro.insert(1,"TIENDA","CLARO SHOP")
        df_claro["PRECIO"]=df_claro["PRECIO"].str.replace("$"," ")

        self.df_to_excel(df_claro,"claro_shop",producto)

        return df_claro


    def scrapping_game_planet(self,producto:str) -> pd:
        """Web scrapper para el sitio de Game Planet.

        Args:
            producto (str): producto del que se desea obtener dataframe

        Returns:
            pd: dataframe del producto
        """
        driver = webdriver.Chrome(service=Service("C://Users//basal//Downloads//chromedriver_win32//chromedriver.exe"))
        driver.implicitly_wait(5)
        producto=producto.replace(" ","%20")
        driver.get(f"https://gameplanet.com/?s={producto}&post_type=product&dgwt_wcas=1")
        driver.implicitly_wait(5)
        productos=driver.find_elements(By.CLASS_NAME,"product-small.box")

        lista_nombres=self.list_maker_class(len(productos),productos,"name.product-title.woocommerce-loop-product__title")
        lista_categorias=self.list_maker_class(len(productos),productos,"category.uppercase.is-smaller.no-text-overflow.product-cat.op-7")
        lista_estados=self.list_maker_class(len(productos),productos,"condicion.uppercase.is-smaller.no-text-overflow.product-condicion")
        lista_url_imagenes=self.list_maker_attribute(len(productos),productos,"img","src")
        lista_urls_productos=self.list_maker_attribute(len(productos),productos,"a","href")

        data={"CATEGORIAS":lista_categorias,"PRODUCTO":lista_nombres,"CONDICION":lista_estados,"IMAGEN":lista_url_imagenes,"LINK":lista_urls_productos}

        df_game_planet=pd.DataFrame(data)
        df_game_planet.insert(0,"FECHA",datetime.now().strftime("%d/%m/%Y"))
        df_game_planet.insert(1,"TIENDA","GAME PLANET")
        
        self.df_to_excel(df_game_planet,"game_planet",producto)

        df_game_planet
        display(HTML(df_game_planet.to_html(escape=False,formatters=dict(IMAGEN=self.imagen))))


In [7]:
WS_prueba7=Web_Scrapper()

WS_prueba7.scrapping_claro("nintendo")

C:\Users\basal\AppData\Local\Temp\ipykernel_3616\4062215563.py:191: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_claro["PRECIO"]=df_claro["PRECIO"].str.replace("$"," ")


,FECHA,TIENDA,PRODUCTO,PRECIO,ENVIO,LINK
0,16/12/2022,CLARO SHOP,Control Nintendo Switch Pro Controller,"1,999",1,https://www.claroshop.com/producto/15017111/co...
1,16/12/2022,CLARO SHOP,Consola Nintendo Switch 1.1 Neon,"8,799",1,https://www.claroshop.com/producto/15028691/co...
2,16/12/2022,CLARO SHOP,Consola Nintendo Switch Oled White 64gb,"9,299",1,https://www.claroshop.com/producto/15017110/co...
3,16/12/2022,CLARO SHOP,Control Nintendo Switch Joy-con L R Neon,"2,199",1,https://www.claroshop.com/producto/15017112/co...
4,16/12/2022,CLARO SHOP,Videojuego NSW Animal Crossing New Horizons,"1,359",1,https://www.claroshop.com/producto/15017105/vi...
...,...,...,...,...,...,...
95,16/12/2022,CLARO SHOP,Soporte de para Nintendo Switch,135,0,https://www.claroshop.com/producto/15534771/so...
96,16/12/2022,CLARO SHOP,Nintendo Switch Pokemon Brilliant Diamond,"1,199",1,https://www.claroshop.com/producto/14142494/ni...
97,16/12/2022,CLARO SHOP,Burgertime Party! - Nintendo Switch,829,1,https://www.claroshop.com/producto/13921442/bu...
98,16/12/2022,CLARO SHOP,Control Joy-Con neón para nintendo switch,"2,141",1,https://www.claroshop.com/producto/919712/cont...


In [10]:
WS_prueba8=Web_Scrapper()

WS_prueba8.scrapping_claro("gears of war 5 xbox one")

C:\Users\basal\AppData\Local\Temp\ipykernel_3616\4062215563.py:191: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_claro["PRECIO"]=df_claro["PRECIO"].str.replace("$"," ")


,FECHA,TIENDA,PRODUCTO,PRECIO,ENVIO,LINK
0,16/12/2022,CLARO SHOP,GEARS OF WAR TOMO BRILLANTE,399,0,https://www.claroshop.com/producto/14747298/ge...
1,16/12/2022,CLARO SHOP,Gears Of War Tomo Mate,399,0,https://www.claroshop.com/producto/15745008/ge...
2,16/12/2022,CLARO SHOP,Gears of War 4 Codigo Digital,449,0,https://www.claroshop.com/producto/15429303/ge...
3,16/12/2022,CLARO SHOP,Alfombrilla Para Mouse Razer RGB Edición Gears...,"2,420",1,https://www.claroshop.com/producto/13782087/al...
4,16/12/2022,CLARO SHOP,Gears Of War 4 En 4k X Box One - Game Center,439,0,https://www.claroshop.com/producto/15056019/ge...
...,...,...,...,...,...,...
95,16/12/2022,CLARO SHOP,Gold Edition de Call of Duty: Advanced Warfare...,899,1,https://www.claroshop.com/producto/15378528/go...
96,16/12/2022,CLARO SHOP,DARK SOULS 2: Scholar of the First Sin Codigo ...,349,0,https://www.claroshop.com/producto/15378540/da...
97,16/12/2022,CLARO SHOP,Call of Duty: Modern Warfare Remastered Codigo...,849,1,https://www.claroshop.com/producto/15378534/ca...
98,16/12/2022,CLARO SHOP,Call of Duty: Modern Warfare II - Lote Multige...,"1,479",1,https://www.claroshop.com/producto/15644364/ca...


In [11]:
WS_prueba9=Web_Scrapper()

WS_prueba9.scrapping_claro("halo infinite xbox")

C:\Users\basal\AppData\Local\Temp\ipykernel_3616\4062215563.py:191: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_claro["PRECIO"]=df_claro["PRECIO"].str.replace("$"," ")


,FECHA,TIENDA,PRODUCTO,PRECIO,ENVIO,LINK
0,16/12/2022,CLARO SHOP,"Halo Infinite, para Xbox One","1,870",1,https://www.claroshop.com/producto/15308779/ha...
1,16/12/2022,CLARO SHOP,Xbox One Halo Infinite,"1,349",1,https://www.claroshop.com/producto/14461255/xb...
2,16/12/2022,CLARO SHOP,Halo Infinite Xbox One / Series X,"1,449",1,https://www.claroshop.com/producto/14631022/ha...
3,16/12/2022,CLARO SHOP,Xbox Serie S Y X Halo Infinite,"1,499",1,https://www.claroshop.com/producto/14628133/xb...
4,16/12/2022,CLARO SHOP,Halo Infinite Edición Estándar - Xbox Series X...,"1,099",1,https://www.claroshop.com/producto/15531771/ha...
...,...,...,...,...,...,...
95,16/12/2022,CLARO SHOP,Cerrojo Electrónico Touch Inteligente WI-FI KW...,"8,499",1,https://www.claroshop.com/producto/15076251/ce...
96,16/12/2022,CLARO SHOP,Espada Halo Energy Sword Master Chief Espuma P...,997,1,https://www.claroshop.com/producto/14340007/es...
97,16/12/2022,CLARO SHOP,Cuadro Canva Decorativo Master Chief Halo 90X6...,909,1,https://www.claroshop.com/producto/13660121/cu...
98,16/12/2022,CLARO SHOP,Extensiones Cabello De Liga / Halo De Fibra Re...,459,0,https://www.claroshop.com/producto/15602118/ex...
